# Part 1: Fetching, Cleaning, & Storing Stock Data in Azure Cloud Data Warehouse

**Step 1: Install and Load Packages**

In [67]:
%pip install pandas numpy yfinance Fred fredapi xgboost scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [126]:
# Import libraries
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
from fredapi import Fred
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss, f1_score, classification_report
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.frozen import FrozenEstimator
import xgboost as xgb
import inspect

**Step 2: Load & Clean Stock Data from API**

In [127]:
def fetch_multiple_stocks(ticker_list, start_date="2021-01-01"):
    df = yf.download(
        ticker_list,
        start=start_date,
        auto_adjust=False,
        group_by="ticker",
        progress=False
    )

    df = (
        df.stack(level=0)
          .reset_index()
          .rename(columns={"level_1": "Ticker"})
    )

    df["Date"] = df["Date"].dt.strftime("%Y-%m-%d")

    df = df[["Ticker", "Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]]
    df = df.sort_values(["Ticker", "Date"]).reset_index(drop=True)

    return df


start_date = "2021-01-01"

train_tickers = [
    "SRFM","RR","THAR",
    "GOOGL","NVDA","MSFT","AAPL","AMZN","META","TSLA","NFLX","ADBE","ORCL",
    "CAVA","CMG","SHAK","WING","DPZ",
    "SYM","PATH","IRBT","TER","ROK","CGNX","FANUY",
    "REGN","MRNA","BMY"
]

report_tickers = ["SRFM","RR","THAR","SOFI","QUBT","RGTI","IONQ"]

context_tickers = [
    "^GSPC","^VIX",
    "QQQ","IWM",
    "XLK","XLF","XLE","XLV","XLY","XLP","XLI","XLC","XLB","XLU",
    "SMH","XBI"
]

all_tickers = sorted(set(train_tickers + report_tickers + context_tickers))

stock_data = fetch_multiple_stocks(all_tickers, start_date)

stock_data.tail(10)


Price,Ticker,Date,Open,High,Low,Close,Adj Close,Volume
58080,^VIX,2025-12-26,14.12,14.29,13.52,13.60,13.60,0.0
58081,^VIX,2025-12-29,14.69,15.08,13.99,14.20,14.20,0.0
58082,^VIX,2025-12-30,14.43,14.62,14.04,14.33,14.33,0.0
58083,^VIX,2025-12-31,14.77,15.17,14.38,14.95,14.95,0.0
58084,^VIX,2026-01-02,14.85,15.42,14.46,14.51,14.51,0.0
58085,^VIX,2026-01-05,15.14,15.27,14.65,14.90,14.90,0.0
58086,^VIX,2026-01-06,15.00,15.21,14.63,14.75,14.75,0.0
58087,^VIX,2026-01-07,14.95,15.48,14.83,15.38,15.38,0.0
58088,^VIX,2026-01-08,15.69,15.85,15.15,15.45,15.45,0.0
58089,^VIX,2026-01-09,15.68,15.81,14.88,15.08,15.08,0.0


**Economic Variables: Interest Rate & Inflation Rate & GDP & Unemployment Rate (USA - Monthly)**

In [128]:
fred = Fred(api_key="63a2c4976cc92b8965e3e45090f491b9")

def fetch_fred_series(start_date="2021-01-01"):
    """
    Fetch and combine core US macro series from FRED.

    Returns:
    - DataFrame with columns:
      Date, Interest_Rate, Inflation_Rate, Unemployment_Rate,
    """
    series_map = {
        "FEDFUNDS": "Interest_Rate",
        "CPIAUCSL": "Inflation_Rate",
        "UNRATE": "Unemployment_Rate"
        }

    dfs = []

    for code, new_name in series_map.items():
        s = fred.get_series(code, observation_start=start_date)
        df = pd.DataFrame({
            "Date": s.index,
            new_name: s.values
            
        })
        dfs.append(df)

    fred_data = dfs[0]
    for df in dfs[1:]:
        fred_data = fred_data.merge(df, on="Date", how="outer")

    fred_data = fred_data.sort_values("Date")
    fred_data["Date"] = pd.to_datetime(fred_data["Date"]).dt.strftime("%Y-%m-%d")
    fred_data.reset_index(drop=True, inplace=True)

    return fred_data

# Example usage
fred_data = fetch_fred_series(start_date="2021-01-01")
fred_data.tail(10)


,Date,Interest_Rate,Inflation_Rate,Unemployment_Rate
50,2025-03-01,4.33,319.615,4.2
51,2025-04-01,4.33,320.321,4.2
52,2025-05-01,4.33,320.580,4.3
53,2025-06-01,4.33,321.500,4.1
54,2025-07-01,4.33,322.132,4.3
55,2025-08-01,4.33,323.364,4.3
56,2025-09-01,4.22,324.368,4.4
57,2025-10-01,4.09,NaN,NaN
58,2025-11-01,3.88,325.031,4.5
59,2025-12-01,3.72,NaN,4.4


**Step 3: Gradient Boosting Model For Stock Price Prediction**

In [149]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, log_loss, f1_score

HORIZON = 15
SIGNAL_DATE_PAST = pd.Timestamp("2025-12-24")

macro_cols = ["Interest_Rate","Inflation_Rate","Unemployment_Rate"]
TARGET_TICKERS = ["NVDA","ORCL","THAR","SOFI","RR","RGTI"]
SPX_TICKER = "^GSPC"
VIX_TICKER = "^VIX"

df = stock_data.copy()
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values(["Ticker","Date"]).reset_index(drop=True)

macro = fred_data.copy()
macro["Date"] = pd.to_datetime(macro["Date"])
macro = macro.sort_values("Date").reset_index(drop=True)

df = df[df["Ticker"].isin(TARGET_TICKERS + [SPX_TICKER, VIX_TICKER])].copy()

def context_series(dfx, tkr):
    col = "Close" if tkr == VIX_TICKER else "Adj Close"
    px = dfx[dfx["Ticker"] == tkr][["Date", col]].copy()
    return px.rename(columns={col: f"PX_{tkr}"})

df = df.merge(context_series(df, SPX_TICKER), on="Date", how="left")
df = df.merge(context_series(df, VIX_TICKER), on="Date", how="left")

df = df[df["Ticker"].isin(TARGET_TICKERS)].copy()
df = df.sort_values("Date").reset_index(drop=True)

df = pd.merge_asof(
    df,
    macro[["Date"] + macro_cols].sort_values("Date"),
    on="Date",
    direction="backward"
)

df = df.sort_values(["Ticker","Date"]).reset_index(drop=True)

df[f"PX_{SPX_TICKER}"] = df[f"PX_{SPX_TICKER}"].ffill()
df[f"PX_{VIX_TICKER}"] = df[f"PX_{VIX_TICKER}"].ffill()
for c in macro_cols:
    df[c] = df[c].ffill()

def make_features(x):
    x = x.sort_values("Date").copy()
    px = x["Adj Close"].astype(float)
    logp = np.log(px)

    x["r1"] = logp.diff()
    x["r5"] = logp.diff(5)
    x["r15"] = logp.diff(15)
    x["r30"] = logp.diff(30)

    r1 = x["r1"]
    x["mom10"] = r1.rolling(10).mean()
    x["mom20"] = r1.rolling(20).mean()
    x["vol20"] = r1.rolling(20).std()

    x["dd30"] = px / px.rolling(30).max() - 1.0

    spx_px = x[f"PX_{SPX_TICKER}"].astype(float)
    spx_log = np.log(spx_px)
    x["spx_r1"] = spx_log.diff()
    x["spx_r30"] = spx_log.diff(30)

    vix = x[f"PX_{VIX_TICKER}"].astype(float)
    x["vix_chg"] = np.log(vix).diff()
    x["vix_lvl"] = vix

    vol = x["Volume"].astype(float)
    x["vol_ratio20"] = vol / vol.rolling(20).mean()

    delta = px.diff()
    up = delta.clip(lower=0)
    down = (-delta).clip(lower=0)
    rs = up.rolling(14).mean() / down.rolling(14).mean()
    x["rsi14"] = 100 - (100 / (1 + rs))

    x["rel_str30"] = x["r30"] - x["spx_r30"]

    dow = x["Date"].dt.dayofweek.astype(float)
    x["dow_sin"] = np.sin(2*np.pi*dow/7.0)
    x["dow_cos"] = np.cos(2*np.pi*dow/7.0)

    x["spx_vol20"] = x["spx_r1"].rolling(20).std()
    x["risk_off"] = (x["vix_lvl"] > x["vix_lvl"].rolling(60).median()).astype(float)

    return x

df = df.groupby("Ticker", group_keys=False).apply(make_features).reset_index(drop=True)

open_dates = df["Date"].drop_duplicates().sort_values().reset_index(drop=True)

def next_open_date(target_date):
    future = open_dates[open_dates >= target_date]
    if len(future) == 0:
        return None
    return future.iloc[0]

def label_3class(r):
    if r <= -0.01:
        return 0
    if r >= 0.01:
        return 2
    return 1

feature_cols = [
    "r1","r5","r15","r30",
    "mom10","mom20","vol20",
    "dd30",
    "spx_r1",
    "vix_chg","vix_lvl",
    "vol_ratio20",
    "rsi14",
    "rel_str30",
    "dow_sin","dow_cos",
    "spx_vol20",
    "risk_off"
] + macro_cols

df["y"] = np.nan
df["y_date"] = pd.NaT

for tkr in TARGET_TICKERS:
    w = df[df["Ticker"] == tkr].copy().sort_values("Date")
    date_to_close = dict(zip(w["Date"].values, w["Adj Close"].values))
    for idx, d in zip(w.index, w["Date"].values):
        tgt = next_open_date(pd.Timestamp(d) + pd.Timedelta(days=HORIZON))
        if tgt is None:
            continue
        if tgt not in date_to_close:
            continue
        cur = float(df.loc[idx, "Adj Close"])
        future = float(date_to_close[tgt])
        r = (future / cur) - 1.0
        df.loc[idx, "y"] = label_3class(r)
        df.loc[idx, "y_date"] = tgt

must_have = ["r1","vol20","spx_r1","vix_lvl","vol_ratio20","rsi14","rel_str30"] + macro_cols

def make_model():
    return XGBClassifier(
        n_estimators=225,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        min_child_weight=5,
        objective="multi:softprob",
        num_class=3,
        eval_metric="mlogloss",
        tree_method="hist",
        random_state=42,
        n_jobs=-1
    )

def print_metrics(title, y_true, proba):
    y_true = np.asarray(y_true, dtype=int)
    y_pred = np.argmax(proba, axis=1)
    print(title)
    print(f"  accuracy    : {accuracy_score(y_true, y_pred):.3f}")
    print(f"  log_loss    : {log_loss(y_true, proba, labels=[0,1,2]):.3f}")
    print(f"  f1_macro    : {f1_score(y_true, y_pred, average='macro'):.3f}")
    print(f"  f1_weighted : {f1_score(y_true, y_pred, average='weighted'):.3f}")

def class_counts_3(y):
    y = np.asarray(y, dtype=int)
    c = np.bincount(y, minlength=3)
    return int(c[0]), int(c[1]), int(c[2])

def backtest_single_ticker(
    ticker,
    start_test_date="2025-01-01",
    step=5,
    min_train_rows=250,
    confidence_threshold=0.6
):
    w = df[df["Ticker"] == ticker].copy().sort_values("Date").reset_index(drop=True)
    start_test_date = pd.to_datetime(start_test_date)

    candidate = w[w["Date"] >= start_test_date].copy()
    candidate = candidate[candidate[must_have].notna().all(axis=1)].copy()

    test_dates = candidate["Date"].drop_duplicates().sort_values().iloc[::step]

    rows_all = []
    y_true_all = []
    y_pred_all = []
    proba_all = []

    for d in test_dates:
        train = w[(w["Date"] < d) & (w["y_date"] <= d)].dropna(subset=feature_cols + ["y"])
        if len(train) < min_train_rows:
            continue

        test = w[w["Date"] == d].dropna(subset=feature_cols)
        if len(test) != 1:
            continue

        X_tr = train[feature_cols].values
        y_tr = train["y"].astype(int).values

        classes = np.unique(y_tr)
        if len(classes) < 2:
            continue

        weights = compute_class_weight(class_weight="balanced", classes=classes, y=y_tr)
        w_map = {int(c): float(wt) for c, wt in zip(classes, weights)}
        sample_w = np.array([w_map[int(yy)] for yy in y_tr], dtype=float)

        model = make_model()
        model.fit(X_tr, y_tr, sample_weight=sample_w)

        proba = model.predict_proba(test[feature_cols].values)
        y_pred = int(np.argmax(proba, axis=1)[0])

        tgt = next_open_date(pd.to_datetime(d) + pd.Timedelta(days=HORIZON))
        if tgt is None:
            continue

        future_row = w[w["Date"] == tgt]
        if len(future_row) != 1:
            continue

        cur = float(test["Adj Close"].iloc[0])
        fut = float(future_row["Adj Close"].iloc[0])
        y_true = label_3class((fut / cur) - 1.0)

        out = test[["Date","Ticker","Adj Close"]].copy().reset_index(drop=True)
        out["Target_Date"] = tgt
        out["Down"] = float(proba[0,0])
        out["Flat"] = float(proba[0,1])
        out["Up"] = float(proba[0,2])
        out["y_true"] = y_true
        out["y_pred"] = y_pred

        max_prob = float(np.max(proba))
        if max_prob >= confidence_threshold:
            if y_pred == 0:
                out["Action"] = "SHORT"
            elif y_pred == 2:
                out["Action"] = "BUY"
            else:
                out["Action"] = "HOLD"
        else:
            out["Action"] = "HOLD"

        rows_all.append(out)
        y_true_all.append(y_true)
        y_pred_all.append(y_pred)
        proba_all.append(proba)

    if len(rows_all) == 0:
        raise ValueError(f"{ticker}: produced no folds. Try earlier start_test_date or lower min_train_rows.")

    bt = pd.concat(rows_all, ignore_index=True)
    y_true_arr = np.array(y_true_all, dtype=int)
    y_pred_arr = np.array(y_pred_all, dtype=int)
    proba_arr = np.vstack(proba_all)

    true_down, true_flat, true_up = class_counts_3(y_true_arr)
    pred_down, pred_flat, pred_up = class_counts_3(y_pred_arr)

    print(f"\n{ticker} — Horizon {HORIZON} cal-days | folds={len(rows_all)} | start={start_test_date.date()} | step={step}")
    print(f"  class counts (true): Down={true_down} Flat={true_flat} Up={true_up}")
    print(f"  class preds  (pred): Down={pred_down} Flat={pred_flat} Up={pred_up}")
    print_metrics("  metrics:", y_true_arr, proba_arr)

    return bt

def backtest_all(start_test_date="2025-01-01", step=5):
    parts = []
    for tkr in TARGET_TICKERS:
        parts.append(backtest_single_ticker(tkr, start_test_date=start_test_date, step=step))
    return pd.concat(parts, ignore_index=True)

def signal_table(signal_date, confidence_threshold=0.6, min_train_rows=250):
    signal_date = pd.to_datetime(signal_date)
    rows = []

    for tkr in TARGET_TICKERS:
        w = df[df["Ticker"] == tkr].copy().sort_values("Date").reset_index(drop=True)

        test = w[w["Date"] == signal_date].dropna(subset=feature_cols)
        if len(test) != 1:
            continue

        train = w[(w["Date"] < signal_date) & (w["y_date"] <= signal_date)].dropna(subset=feature_cols + ["y"])
        if len(train) < min_train_rows:
            continue

        X_tr = train[feature_cols].values
        y_tr = train["y"].astype(int).values

        classes = np.unique(y_tr)
        if len(classes) < 2:
            continue

        weights = compute_class_weight(class_weight="balanced", classes=classes, y=y_tr)
        w_map = {int(c): float(wt) for c, wt in zip(classes, weights)}
        sample_w = np.array([w_map[int(yy)] for yy in y_tr], dtype=float)

        model = make_model()
        model.fit(X_tr, y_tr, sample_weight=sample_w)

        proba = model.predict_proba(test[feature_cols].values)[0]

        tgt = next_open_date(signal_date + pd.Timedelta(days=HORIZON))
        if tgt is None:
            tgt = pd.NaT

        max_prob = float(np.max(proba))
        pred_class = int(np.argmax(proba))

        if max_prob >= confidence_threshold:
            action = "SHORT" if pred_class == 0 else "BUY" if pred_class == 2 else "HOLD"
        else:
            action = "HOLD"

        rows.append({
            "Date": signal_date,
            "Ticker": tkr,
            "Adj Close": float(test["Adj Close"].iloc[0]),
            "Target_Date": tgt,
            "Down": float(proba[0]),
            "Flat": float(proba[1]),
            "Up": float(proba[2]),
            "Action": action
        })

    out = pd.DataFrame(rows)
    if len(out) > 0:
        out = out.sort_values("Ticker").reset_index(drop=True)
    return out

bt = backtest_all(start_test_date="2025-01-01", step=5)

print("\n=== Backtest Evaluation Complete ===")
print(f"Total predictions: {len(bt)}")

signal_date_today = df["Date"].max()

table_past = signal_table(SIGNAL_DATE_PAST, confidence_threshold=0.6)
table_now = signal_table(signal_date_today, confidence_threshold=0.6)

print("\n=== Signals on 2025-12-24 ===")
display(table_past[["Date","Ticker","Adj Close","Target_Date","Down","Flat","Up","Action"]])

print("\n=== Signals on MOST RECENT DATE ===")
display(table_now[["Date","Ticker","Adj Close","Target_Date","Down","Flat","Up","Action"]])



NVDA — Horizon 15 cal-days | folds=50 | start=2025-01-01 | step=5
  class counts (true): Down=19 Flat=4 Up=27
  class preds  (pred): Down=21 Flat=4 Up=25
  metrics:
  accuracy    : 0.480
  log_loss    : 1.189
  f1_macro    : 0.342
  f1_weighted : 0.483

ORCL — Horizon 15 cal-days | folds=50 | start=2025-01-01 | step=5
  class counts (true): Down=23 Flat=1 Up=26
  class preds  (pred): Down=25 Flat=1 Up=24
  metrics:
  accuracy    : 0.380
  log_loss    : 1.247
  f1_macro    : 0.258
  f1_weighted : 0.380

THAR — Horizon 15 cal-days | folds=50 | start=2025-01-01 | step=5
  class counts (true): Down=28 Flat=4 Up=18
  class preds  (pred): Down=37 Flat=1 Up=12
  metrics:
  accuracy    : 0.440
  log_loss    : 1.178
  f1_macro    : 0.373
  f1_weighted : 0.407

SOFI — Horizon 15 cal-days | folds=50 | start=2025-01-01 | step=5
  class counts (true): Down=16 Flat=3 Up=31
  class preds  (pred): Down=12 Flat=2 Up=36
  metrics:
  accuracy    : 0.540
  log_loss    : 1.147
  f1_macro    : 0.324
  f1_w

,Date,Ticker,Adj Close,Target_Date,Down,Flat,Up,Action
0,2025-12-24,NVDA,188.610001,2026-01-08,0.234520,0.475599,0.289880,HOLD
1,2025-12-24,ORCL,197.490005,2026-01-08,0.870308,0.062953,0.066739,SHORT
2,2025-12-24,RGTI,24.510000,2026-01-08,0.356186,0.023760,0.620053,BUY
3,2025-12-24,RR,3.650000,2026-01-08,0.704402,0.017334,0.278264,SHORT
4,2025-12-24,SOFI,27.480000,2026-01-08,0.489156,0.055708,0.455136,HOLD
5,2025-12-24,THAR,2.560000,2026-01-08,0.904759,0.015059,0.080182,SHORT



=== Signals on MOST RECENT DATE ===


,Date,Ticker,Adj Close,Target_Date,Down,Flat,Up,Action
0,2026-01-09,NVDA,185.383606,NaT,0.718856,0.049386,0.231759,SHORT
1,2026-01-09,ORCL,197.360001,NaT,0.787615,0.026498,0.185887,SHORT
2,2026-01-09,RGTI,25.355000,NaT,0.298078,0.033947,0.667975,BUY
3,2026-01-09,RR,3.820000,NaT,0.409744,0.009108,0.581148,HOLD
4,2026-01-09,SOFI,28.150000,NaT,0.573704,0.066516,0.359780,HOLD
5,2026-01-09,THAR,2.900000,NaT,0.970221,0.013702,0.016077,SHORT
